<a href="https://colab.research.google.com/github/poliduckie/poliduckie_pipeline/blob/master/notebooks/poliduckie_mpc_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PoliDuckies
A notebook with the complete pipeline

### Standard imports

In [ ]:
import os 
branch = "daffy" #@param ['master', 'daffy']
if branch == "daffy":
  !pip install duckietown-gym-daffy
  !pip install pyglet==1.5.15
else:
  if not os.path.isdir('gym-duckietown') and not os.path.isdir('../gym-duckietown'):
    !git clone --branch {branch} https://github.com/duckietown/gym-duckietown.git
    !pip3 install -e gym-duckietown
  if "/gym-duckietown" not in os.getcwd():
    os.chdir('gym-duckietown')
!apt install xvfb -y
!pip3 install pyvirtualdisplay

from pyvirtualdisplay import Display
import numpy as np
import matplotlib.pyplot as plt
from IPython import display as ipythondisplay
%matplotlib inline

## used to copy the model to the gdrive folder
import shutil

import cv2
import gym
import gym_duckietown

# Gym has a built in logger to log information. The information is split in importance levels, from highest to lowest:
# CRITICAL, ERROR, WARNING, INFO, DEBUG, NOTSET
# To stop gym from printing many verbose NOTSET, INFO and DEBUG messages we let it print only level WARNING or higher messages
import logging
gym_duckietown.logger.setLevel(logging.WARNING)

!pip install poliduckie-segmentation

In [ ]:
map_name = "Duckietown-ETH_large_loop"

### Methods

In [ ]:
from poliduckie_segmentation.segmentation import Segmentation
def segmentation(image):
  """
    Returns the image from the simulation with the segmentation

    Image: np.Array
    ------
    Image: np.Array
  """

  seg = Segmentation()
  return seg.predict(image)

In [4]:
from poliduckie_segmentation.line_extraction import LineExtraction
def lineExtraction(image, n_points=100, mode="bezier", bezier_degree=3):
  """
    Returns the central line extracted as a numpy array. The mode can be either "bezier" or "spline"

    Image: np.Array
    n_points: int
    mode: str
    bezier_degree: int
    ------
    Line: np.Array
  """
  lineExtraction_ = LineExtraction()
  x, y, pca = lineExtraction_.extract_centroids(image)

  if mode == "bezier":
    bezier_points = lineExtraction_.fit_bezier(x, y, degree=bezier_degree, n_points=n_points)
    return bezier_points
  
  elif mode == "spline":
    spline_points = lineExtraction_.spline_interpolation(image, n_points=n_points)
    return spline_points

In [ ]:
def computeReference(line, N):
  """
    Returns the references in meters
    Reference is a list with dimension (2, N)
    N is the prediction horizon of the MPC

    Line: np.Array
    N: int
    ------
    reference: List[[List[Float], List[Float]]]
  """
  # extract x, y of reference
  # TODO

  reference = [[x]*N, [y]*N]
  pass

In [ ]:
from poliduckie_segmentation.control import MPC
M = MPC()

def runMpc(state, reference):
  """
    Returns the control inputs to give to the car.
    State is [x, y, theta, v, w]
    Reference is a list with dimension (2, N)

    state: [Float, Float, Float, Float, Float]
    reference: List[[List[Float], List[Float]]]
    ------
    u: List[Float, Float]
  """
  return M.mpc(state, reference)

### Simulation

In [ ]:
display = Display(visible=0, size=(1400, 900))
display.start()
env = gym.make(map_name)
env.reset()

In [ ]:
from poliduckie_segmentation.model import Model

# State X = [x, y, angle, longitudinal speed, angular speed]
state = [0,0,0,0,0]

F = Model()

image_recording = []

for _ in range(100):
  image = env.render(mode="rgb_array")
  image_recording.append(image)
  image_segmentation = segmentation(image)
  line = lineExtraction(image_segmentation)
  reference = computeReference(line)
  action = runMpc(state, reference)
  state = F.step(*state, action)
  env.step(action)